# Apfelpy
Test TMD wrapping functions.

In [ ]:
import numpy as np
import lhapdf as lh
import apfelpy as ap
import matplotlib.pyplot as plt

## hard factors

In [ ]:
### --- Test the hard factor functions

print("H1DY =", ap.hardFactors.H1DY())
print("H2DY(nf=5) =", ap.hardFactors.H2DY(5))
print("H3DY(nf=5) =", ap.hardFactors.H3DY(5))

print("H1SIDIS =", ap.hardFactors.H1SIDIS())
print("H2SIDIS(nf=5) =", ap.hardFactors.H2SIDIS(5))
print("H3SIDIS(nf=5) =", ap.hardFactors.H3SIDIS(5))

print("H3Ch =", ap.hardFactors.H3Ch())
print("H1ggH =", ap.hardFactors.H1ggH())
print("H2ggH(nf=5) =", ap.hardFactors.H2ggH(5))

## TMDs

### TMD object initializers

In [ ]:
# Create a pdf grid for TMD functions
# Grid pdfs
gpdf = ap.Grid([ap.SubGrid(100, 1e-4, 3), ap.SubGrid(60,1e-1,3), ap.SubGrid(50,6e-1,3), ap.SubGrid(50,8e-1,3)])

# Define thresholds and masses
thresholds = [0, 0, 0, np.sqrt(2), 4.5, 175]

print("\n *** 1. Testing TMD Object Initializers ***\n")

# Test different TMD object initializers
print("- Testing InitializeTmdObjects")
tmd_objs = ap.tmd.InitializeTmdObjects(gpdf, thresholds)
print(f"  Created objects for {len(tmd_objs)} active flavor numbers")


print("- Testing InitializeTmdObjectsDYResScheme")
tmd_objs_dy = ap.tmd.InitializeTmdObjectsDYResScheme(gpdf, thresholds)
print(f"  Created DY scheme objects for {len(tmd_objs_dy)} active flavor numbers")


print("- Testing InitializeTmdObjectsBM")
tmd_objs_bm = ap.tmd.InitializeTmdObjectsBM(gpdf, thresholds)
print(f"  Created BM objects for {len(tmd_objs_bm)} active flavor numbers")


print("- Testing InitializeTmdObjectsSivers")
tmd_objs_sivers = ap.tmd.InitializeTmdObjectsSivers(gpdf, thresholds)
print(f"  Created Sivers objects for {len(tmd_objs_sivers)} active flavor numbers")


print("- Testing InitializeTmdObjectsg1")
tmd_objs_g1 = ap.tmd.InitializeTmdObjectsg1(gpdf, thresholds)
print(f"  Created g1 objects for {len(tmd_objs_g1)} active flavor numbers")



In [ ]:
### --- Look inside one TMD object --- ###

# Get the one with 5 active flavors
test_obj = tmd_objs[5]
print("\n *** Examining TmdObjects structure ***\n")

print(f"- Threshold: {test_obj.Threshold}")
print(f"- Beta keys: {list(test_obj.Beta.keys())}")
print(f"- GammaFq keys: {list(test_obj.GammaFq.keys())}")
print(f"- GammaFg keys: {list(test_obj.GammaFg.keys())}")
print(f"- GammaK keys: {list(test_obj.GammaK.keys())}")
print(f"- KCS keys: {list(test_obj.KCS.keys())}")
print(f"- MatchingFunctionsPDFs keys: {list(test_obj.MatchingFunctionsPDFs.keys())}")
print(f"- MatchingFunctionsFFs keys: {list(test_obj.MatchingFunctionsFFs.keys())}")
print(f"- HardFactors keys: {list(test_obj.HardFactors.keys())}")



### TMD builders

#### setup

In [ ]:
### --- Setup for testing the TMD functions --- ###

# ----------------------------------------------
# Initalise LHAPDF set
# ----------------------------------------------
pdfs = lh.mkPDF("MMHT2014nnlo68cl", 0)

# ----------------------------------------------
# Define thresholds and masses
# ----------------------------------------------
thresholds = [0, 0, 0, np.sqrt(2), 4.5, 175]
masses     = [0, 0, 0, np.sqrt(2), 4.5]

# ----------------------------------------------
# Alpha_s function
# ----------------------------------------------
# Define alpha_s function using APFEL
alpha_s_apfelpy = ap.AlphaQCD(0.118, 91.2, masses, thresholds, 100)

# Define the Alpha_s function using the LHAPDF set
alpha_s = lambda mu: pdfs.alphasQ(mu)

# Create a TabulateObject (callable) for alpha_s
tab_alphas = ap.TabulateObject(
    alpha_s,                     # Function to tabulate NOTE: can also be the APFEL alpha_s function
    100,                         # Number of points in the mu grid
    np.sqrt(pdfs.q2Min) * 0.9,   # Minimum mu value (QMin)
    np.sqrt(pdfs.q2Max),         # Maximum mu value (QMax)
    3,                           # Interpolation degree
    thresholds                   # Quark mass thresholds
)

# ----------------------------------------------
# Collinear PDF function
# ----------------------------------------------
def coll_pdf_func(mu):
    """
    The calls to TMD builders (later on) will require a callable for the evolved collinear PDFs.
    This function simulates returning a collinear PDF set (SetD) at scale mu = Q.
    Evolve your PDFs from an initial condition.
    """    
    
    # Initial scale
    mu0 = np.sqrt(pdfs.q2Min)

    # Initialize QCD evolution objects
    DglapObj = ap.initializers.InitializeDglapObjectsQCD(gpdf, masses, thresholds)

    # Construct the DGLAP objects
    # Evolve the PDFs: evolve_PDFs is a SetD object returned by BuildDglap
    evolved_PDFs = ap.builders.BuildDglap(
        DglapObj,
        lambda x, mu: ap.utilities.PhysToQCDEv(pdfs.xfxQ(x, mu)),
        mu0, 
        pdfs.orderQCD,
        tab_alphas.Evaluate  # using the Evaluate method of tab_alphas
    )

    # Tabulate the evolved collinear PDFs (returns a TabulateObjectSetD)
    tab_PDFs = ap.TabulateObjectSetD(
        evolved_PDFs, 
        100, 
        np.sqrt(pdfs.q2Min) * 0.9, 
        np.sqrt(pdfs.q2Max), 
        3
    )

    # Tabulate collinear PDFs
    tab_PDFs = ap.TabulateObjectSetD(evolved_PDFs, 100, np.sqrt(pdfs.q2Min) * 0.9, np.sqrt(pdfs.q2Max), 3)
    
    # Directly return the evaluated SetD at scale mu:
    return tab_PDFs.Evaluate(mu)

NOTE:

In the previous cell, a `TabulateObject` is built from the  `alpha_s` callable; this object is intended to tabulate $\alpha_s$.

__Key point__: Instead of passing the entire `tab_alphas` object to `BuildTmdPDFs`, we use its `Evaluate` method (which matches the required signature: it takes a float and returns a float).

#### building functions

In [ ]:
print("\n *** Testing TMD Building Functions ***\n")

# ----------------------------------------------
# Build the TMD PDF function using BuildTmdPDFs
# ----------------------------------------------
# IMPORTANT: The third argument should be a callable returning a float.
# Since tab_alphas is a TabulateObject, we pass its Evaluate method:
tmd_pdf_func = ap.tmd.BuildTmdPDFs(tmd_objs, coll_pdf_func, tab_alphas.Evaluate, 0)

# -----------------------------
# Test the TMD PDF function
# -----------------------------
bT = 0.5         
mu = 91.1876     
zeta = mu * mu  

# Call the TMD PDF function; expect a SetD (collinear PDF set) as return
tmd_pdf_set = tmd_pdf_func(bT, mu, zeta)

# Print information about the returned Distribution set (SetD)
print("Returned object type:", type(tmd_pdf_set))
try:
    # Try using methods of SetD (for example, using 'at')
    print("Number of distributions in the set:", len(tmd_pdf_set))
    # for i in range(len(tmd_pdf_set)):
    #     dist = tmd_pdf_set.at(i)
    #     print(f"Distribution {i}: {dist}")
except Exception as e:
    print("Error accessing the DistributionSet:", e)



#### matching functions

In [ ]:
## -- Test MatchingFunctionsPDFs
matching_pdfs = ap.tmd.MatchingFunctionsPDFs(tmd_objs, lambda mu: alpha_s.Evolve(mu), 1)
print("- MatchingFunctionsPDFs function created")

## -- Test MatchingFunctionsFFs
matching_ffs = ap.tmd.MatchingFunctionsFFs(tmd_objs, lambda mu: alpha_s.Evolve(mu), 1)
print("- MatchingFunctionsFFs function created")



The function `MatchTmdPDFs` has a similar signature to `BuildTmdPDFs` but performs matching in b-space.
Its parameters are:
 - TmdObj: dictionary of TmdObjects,
 - CollPDFs: callable for collinear PDFs (SetD),
 - Alphas: callable for alpha_s (here we use tab_alphas.Evaluate),
 - PerturbativeOrder: an integer,
 - Ci: (default 1).

In [ ]:
# ----------------------------------------------
# Test MatchTmdPDFs: returns a callable for matched TMD PDFs
# ----------------------------------------------
# match_tmd_pdfs = ap.tmd.MatchTmdFFs(tmd_objs, coll_pdf_func, tab_alphas.Evaluate, 0)
match_tmd_pdfs = ap.tmd.MatchTmdPDFs(tmd_objs, coll_pdf_func, tab_alphas.Evaluate, 0)
print("- MatchTmdPDFs function created")

# Define example kinematic parameters:
bT = 0.5         
mu = 91.1876     
zeta = mu * mu   

# Call the returned function to obtain matched TMD PDFs (a SetD)
matched_tmd_pdf_set = match_tmd_pdfs(bT)

print("\n*** Testing MatchTmdPDFs ***")
print("Returned object type (MatchTmdPDFs):", type(matched_tmd_pdf_set))
try:
    # Use available methods from SetD, e.g., using 'at'
    print("Number of distributions in the matched set:", len(matched_tmd_pdf_set))
    for i in range(len(matched_tmd_pdf_set)):
        dist = matched_tmd_pdf_set.at(i)
        print(f"Matched Distribution {i}: {dist}")
except Exception as e:
    print("Error accessing the matched DistributionSet:", e)

# ----------------------------------------------
# Test MatchingFunctionsPDFs: returns matching functions directly (as a SetD)
# ----------------------------------------------
# This function does not require a collinear PDF callable.
# matching_ff_set = ap.tmd.MatchingFunctionsFFs(tmd_objs, tab_alphas.Evaluate, 0)
matching_pdf_set = ap.tmd.MatchingFunctionsPDFs(tmd_objs, tab_alphas.Evaluate, 0)
print("- MatchingFunctionsPDFs function created")

print("\n*** Testing MatchingFunctionsPDFs ***")
print("Returned object type (MatchingFunctionsPDFs):", type(matching_pdf_set))
try:
    print("Number of matching functions in the set:", len(matching_pdf_set))
    for i in range(len(matching_pdf_set)):
        func = matching_pdf_set.at(i)
        print(f"Matching function {i}: {func}")
except Exception as e:
    print("Error accessing the matching functions set:", e)

## TMD evolution factor functions

### test EvolutionFactors

In [ ]:
# --- Setup for testing EvolutionFactors ---
pdfs = lh.mkPDF("MMHT2014nnlo68cl", 0)
thresholds = [0, 0, 0, np.sqrt(2), 4.5, 175]
masses     = [0, 0, 0, np.sqrt(2), 4.5]

# Define alpha_s function using LHAPDF
alpha_s = lambda mu: pdfs.alphasQ(mu)

# Build the evolution factors function
evol_func = ap.tmd.EvolutionFactors(tmd_objs, alpha_s, 0)  # using default Ci and IntEps

# Define kinematic parameters:
bT = 0.5
mu = 91.1876
zeta = mu * mu

# Call the evolution factors function
evolution_factors = evol_func(bT, mu, zeta)
print("\033[92mEvolution factors (list of doubles):", evolution_factors, "\033[0m")


### test EvolutionFactorsK

In [ ]:
# --- Setup for testing EvolutionFactorsK ---
# Create TMD objects
# tmd_objs = ap.tmd.InitializeTmdObjects(gpdf, thresholds)

# Define a simple αₛ function (for testing, just return a constant)
# alpha_s = lambda mu: 0.118

# Set kinematic parameters
bT = 0.5         # transverse distance parameter
mu = 91.1876     # renormalisation scale (e.g., Z mass)
zeta = mu * mu   # rapidity scale (for example, μ²)

# Call EvolutionFactorsK; it returns a callable f(bT, mu, zeta) -> list of doubles.
evolK_func = ap.tmd.EvolutionFactorsK(tmd_objs, alpha_s, 0)
evolution_factors_k = evolK_func(bT, mu, zeta)

print("\033[92mEvolutionFactorsK returned:", evolution_factors_k, "\033[0m")


### test QuarkEvolutionFactor

In [ ]:
# Create TMD objects
# tmd_objs = ap.tmd.InitializeTmdObjects(gpdf, thresholds)

# Simple αₛ function
# alpha_s = lambda mu: 0.118

# Kinematic parameters
bT = 0.5
mu = 91.1876
zeta = mu * mu

# Call QuarkEvolutionFactor; it returns a callable f(bT, mu, zeta) -> double.
q_evol_func = ap.tmd.QuarkEvolutionFactor(tmd_objs, alpha_s, 0)
quark_evol_factor = q_evol_func(bT, mu, zeta)

print("\033[92mQuarkEvolutionFactor returned:", quark_evol_factor, "\033[0m")


### test QuarkEvolutionFactorxi

In [ ]:
# Create TMD objects
# tmd_objs = ap.tmd.InitializeTmdObjects(gpdf, thresholds)

# Simple αₛ function
# alpha_s = lambda mu: 0.118

# Kinematic parameters
bT = 0.5
mu = 91.1876
zeta = mu * mu

# For QuarkEvolutionFactorxi, there is an extra parameter "xi". We use the default value 1.
xi = 1.0

# Call QuarkEvolutionFactorxi; it returns a callable f(bT, mu, zeta) -> double.
q_evol_xi_func = ap.tmd.QuarkEvolutionFactorxi(tmd_objs, alpha_s, 0, xi)
quark_evol_factor_xi = q_evol_xi_func(bT, mu, zeta)

print("\033[92mQuarkEvolutionFactorxi returned:", quark_evol_factor_xi, "\033[0m")


### test GluonEvolutionFactor

In [ ]:
# # Create TMD objects
# tmd_objs = ap.tmd.InitializeTmdObjects(gpdf, thresholds)

# Simple αₛ function
# alpha_s = lambda mu: 0.118

# Kinematic parameters
bT = 0.5
mu = 91.1876
zeta = mu * mu

# Call GluonEvolutionFactor; it returns a callable f(bT, mu, zeta) -> double.
g_evol_func = ap.tmd.GluonEvolutionFactor(tmd_objs, alpha_s, 0)
gluon_evol_factor = g_evol_func(bT, mu, zeta)

print("\033[92mGluonEvolutionFactor returned:", gluon_evol_factor, "\033[0m")


## TMD analytic evolution and kernel functions

### test QuarkAnalyticEvolutionFactor

In [ ]:
# Create TMD objects
tmd_objs = ap.tmd.InitializeTmdObjects(gpdf, thresholds)

# Define necessary parameters:
mu0 = 91.1876           # Example initial scale (e.g., Z boson mass)
Alphas0 = 0.118         # Example αₛ value at mu0
kappa = 1.0             # Example value for kappa (process dependent)
kappa0 = 1.0            # Example value for kappa0 (could be 1.0 as default)
pert_order = 0          # Perturbative order (e.g., 0 for LO)

# Call the wrapped function for quark analytic evolution factor.
# This function is assumed to return a double.
quark_analytic = ap.tmd.QuarkAnalyticEvolutionFactor(tmd_objs[4], mu0, Alphas0, kappa, kappa0, pert_order)

# Now call the returned callable with a specific b value.
b_value = 0.5
result = quark_analytic(b_value)

# Print the result in green.
print("\033[92mQuarkAnalyticEvolutionFactor returned:", result, "\033[0m")


### test GluonAnalyticEvolutionFactor

In [ ]:
# Create TmdObjects
tmd_objs = ap.tmd.InitializeTmdObjects(gpdf, thresholds)

# Define parameters:
mu0     = 91.1876   # Initial scale (e.g., Z boson mass)
Alphas0 = 0.118     # αₛ at mu0
kappa   = 1.0       # Process-dependent parameter (example)
kappa0  = 1.0       # Baseline value for kappa0
pert_order = 0      # Perturbative order

# Call GluonAnalyticEvolutionFactor with a single TmdObjects instance.
gluon_analytic = ap.tmd.GluonAnalyticEvolutionFactor(tmd_objs[4], mu0, Alphas0, kappa, kappa0, pert_order)

# The function returns a callable that accepts one float (for b)
b_value = 0.5
result = gluon_analytic(b_value)

# Print result in green.
print("\033[92mGluonAnalyticEvolutionFactor returned:", result, "\033[0m")


### test CollinsSoperKernel

In [ ]:
# For CollinsSoperKernel we assume the binding expects the full set,
# so we pass a dictionary of TmdObjects.
tmd_objs = ap.tmd.InitializeTmdObjects(gpdf, thresholds)

# Define a simple αₛ function.
alpha_s = lambda mu: 0.118

# Define additional parameters.
pert_order = 0
Ci = 1.0
IntEps = 1e-7

# Call CollinsSoperKernel.
# It returns a callable that accepts two arguments (e.g., bT and Q).
kernel_func = ap.tmd.CollinsSoperKernel(tmd_objs, alpha_s, pert_order, Ci, IntEps)

# Define kinematic parameters:
bT = 0.5    # transverse distance parameter
Q  = 91.1876  # scale (e.g., Z mass)

# Call the kernel function.
kernel_value = kernel_func(bT, Q)

# Print result in green.
print("\033[92mCollinsSoperKernel returned:", kernel_value, "\033[0m")


### test HardFactor

In [ ]:
# For HardFactor we pass a dictionary of TmdObjects.
tmd_objs = ap.tmd.InitializeTmdObjects(gpdf, thresholds)

# Define a simple αₛ function.
alpha_s = lambda mu: 0.118

# Define the process string (for example, "DY" for Drell-Yan).
process = "DY"

# Define perturbative order and color factor Cf (if needed).
pert_order = 0
Cf = 1.0

# Call HardFactor.
# It returns a hard factor as a callable.
hard_factor = ap.tmd.HardFactor(process, tmd_objs, alpha_s, pert_order, Cf)

# The function returns a callable that accepts one float (for Q)
Q_value = 1

# Print result in green.
print("\033[92mHardFactor returned:", hard_factor(Q_value), "\033[0m")
